# Quechua Translator

# 0. Import Analysis Packages

In [1]:
import bs4 
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
import io
import re

# 1. Initial Scraping
I will be using bible data from the following URL - there are many quechua translations, we will be using the one for Quechua from Cuzco for this purpose.
- http://live.bible.is/bible/ENGNAS/MAT/1 - Simplified English Version
- http://live.bible.is/bible/QUZBIV/MAT/1 - Quechua Version

In [2]:
new_testament_codes = pd.read_csv('New_Testatment_URL.csv')
# new_testament_codes = new_testament_codes[:2]
new_testament_codes

,Book,Ending,URL_Alias,Chapter
0,Matthew,MAT/28,MAT,28
1,Mark,MRK/16,MRK,16
2,Luke,LUK/24,LUK,24
3,John,JHN/21,JHN,21
4,Acts,ACT/28,ACT,28
5,Romans,ROM/16,ROM,16
6,1 Corinthians,1CO/16,1CO,16
7,2 Corinthians,2CO/13,2CO,13
8,Galatians,GAL/6,GAL,6
9,Ephesians,EPH/6,EPH,6


In [ ]:
final_data = []
final_data = pd.DataFrame(final_data)

m = len(new_testament_codes)
for j in np.arange(m):
    string_1 = new_testament_codes['URL_Alias'][j]
    string_2 = new_testament_codes['Chapter'][j]
    iterable = string_2
    
    for i in np.arange(iterable): 

        url = "http://live.bible.is/bible/QUZBIV/"
        final_url = f'{url}{string_1}/{i+2}'
        r = requests.get(final_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        text = soup.get_text()
        df = pd.read_csv(io.StringIO(text), sep="\xa0")        
        new_df = df.T.reset_index(drop=False)
        reindexed_df = new_df.reset_index(drop=False)
        last_df = reindexed_df
        last_df['level_0'] = reindexed_df['level_0']+1
        intermediate_df = new_df.reset_index(drop=False)
        final_df = intermediate_df.merge(last_df, on='level_0')
        final_df['Scripture'] =  str(string_1)+"-"+str(i+1)
        final_data = final_data.append(final_df)

C:\Users\MarceloFuentes\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the separator encoded in utf-8 is > 1 char long, and the 'c' engine does not support such separators; you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [ ]:
final_data['Final_Tags'] = final_data['Scripture']+"-"+final_data['index_y'] 
final_data['Filter'] = final_data['Final_Tags'].str[-1]*1
testing_data = final_data
testing_data['Filter'] = testing_data['Filter']
testing_data = testing_data[testing_data['Filter'].str.contains('[0-9]')]
testing_data = testing_data[testing_data['index_y'].str.contains('Bible')==False]
testing_data = testing_data[['index_x','Final_Tags']] 
testing_data

In [ ]:
replacements = {'Ã':'í',
                '±':'ñ',
                'â€':'',
                '~':'',
                '™':'',
                'º':'ú',
                '©':'é',
                'Â':'—',
                '˜':'',
                'œ':'“'}

In [ ]:
testing_data['index_x'] = testing_data['index_x'].replace(replacements) 

In [ ]:
# testing_data[testing_data['Final_Tags']=='MRK-16-1']

In [ ]:
testing_data.to_csv('Quechua_Bible_Data.csv', encoding='utf-8-sig')
# testing_data.to_csv('English_Bible_Data.csv', encoding='utf-8-sig')

# 2. Import Data and Analyze

In [ ]:
# Quechua & English Bible
quechua_bible = pd.read_csv('Quechua_Bible_Data.csv', encoding='utf-8-sig') 
english_bible = pd.read_csv('English_Bible_Data.csv', encoding='utf-8-sig') 

In [ ]:
quechua_bible['Phrase_Length'] = 0
english_bible['Phrase_Length'] = 0


# Explore the data - Length of frases
for i in np.arange(len(quechua_bible)):
    quechua_bible['Phrase_Length'][i] = quechua_bible['index_x'][i].count(' ')+1
    
for j in np.arange(len(english_bible)):
    english_bible['Phrase_Length'][j] = english_bible['index_x'][j].count(' ')+1

In [ ]:
# What is the distribution of phrase lengths in our database?
import matplotlib.pyplot as plt

plt.hist(quechua_bible['Phrase_Length'])
plt.title('Quechua phrases Length Distribution')
plt.show()
plt.hist(english_bible['Phrase_Length'])
plt.title('English phrases Length Distribution')
plt.show()

In [ ]:
# Look at the most repeated words in both dataset
import re
from collections import Counter

quechua_bible_all_words = ' '.join(quechua_bible['index_x'])
english_bible_all_words = ' '.join(english_bible['index_x'])

quechua_words = re.findall(r'\w+', quechua_bible_all_words.lower())
Counter(quechua_words).most_common(40)

english_words = re.findall(r'\w+', english_bible_all_words.lower())
Counter(english_words).most_common(40)

In [ ]:
# Creating Final Dataframe for Translation
english_bible_redux = english_bible[['index_x','Final_Tags']]
quechua_bible_redux = quechua_bible[['index_x','Final_Tags']]
rosetta_bible = english_bible_redux.merge(quechua_bible_redux, on='Final_Tags')
rosetta_bible = rosetta_bible.rename(columns={'index_x_x':'English', 'Final_Tags':'Book_Chapter_Verse', 'index_x_y':'Quechua'})

# converting every letter to lower case
rosetta_bible['Quechua'] = rosetta_bible['Quechua'].apply(lambda x: str(x).lower())
rosetta_bible['English'] = rosetta_bible['English'].apply(lambda x: str(x).lower())

rosetta_bible


# 3. Further Analysis

In [ ]:
import collections
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
# from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

## 3.1. Vocabulary Breakdown

In [ ]:
# English Sentences
english_sentences = rosetta_bible['English']
# Quechua Sentences
quechua_sentences = rosetta_bible['Quechua']


In [ ]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
quechua_words_counter = collections.Counter([word for sentence in quechua_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} Quechua words.'.format(len([word for sentence in quechua_sentences for word in sentence.split()])))
print('{} unique Quechua words.'.format(len(quechua_words_counter)))
print('10 Most common words in the Quechua dataset:')
print('"' + '" "'.join(list(zip(*quechua_words_counter.most_common(10)))[0]) + '"')

# 3.2 Tokenize

In [ ]:
import collections
# from helper import load_data
import numpy as np
# import project_tests as tests
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from tensorflow.keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

In [ ]:
def tokenize(x):
    x_tk = Tokenizer(char_level = False)
    x_tk.fit_on_texts(x)
    return x_tk.texts_to_sequences(x), x_tk


text_tokenized, text_tokenizer = tokenize(english_sentences)
print(text_tokenizer.word_index)

for sample_i, (sent, token_sent) in enumerate(zip(english_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

In [ ]:
def pad(x, length=None):
    if length is None:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen = length, padding = 'post')

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

In [ ]:
def preprocess(x, y):
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)
    
    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)
# Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)
    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_quechua_sentences, english_tokenizer, quechua_tokenizer = preprocess(english_sentences, quechua_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_quechua_sequence_length = preproc_quechua_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
quechua_vocab_size = len(quechua_tokenizer.word_index)
print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max Quechua sentence length:", max_quechua_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("Quechua vocabulary size:", quechua_vocab_size)


# 3.3 IDs back to Text

In [ ]:
def logits_to_text(logits, tokenizer):
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'
    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])
    print('`logits_to_text` function loaded.')

# 4.1 Simple RNN Model

In [46]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, quechua_vocab_size):
    learning_rate = 1e-3
    input_seq = Input(input_shape[1:])
    rnn = GRU(64, return_sequences = True)(input_seq)
    logits = TimeDistributed(Dense(quechua_vocab_size))(rnn)
    model = Model(input_seq, Activation('softmax')(logits))
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    
    return model


# tests.test_simple_model(simple_model)
tmp_x = pad(preproc_english_sentences, max_quechua_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_quechua_sentences.shape[-2], 1))


# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_quechua_sequence_length,
    english_vocab_size,
    quechua_vocab_size)
simple_rnn_model.fit(tmp_x, preproc_quechua_sentences, batch_size=1024, epochs=10, validation_split=0.2)
# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], quechua_tokenizer))

Epoch 1/10
7/7 [==============================] - ETA: 0s - loss: 10.2902 - accuracy: 0.6566  

InvalidArgumentError: Graph execution error:

Detected at node 'sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits' defined at (most recent call last):
    File "C:\Users\MarceloFuentes\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\MarceloFuentes\anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\MarceloFuentes\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\Users\MarceloFuentes\anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\MarceloFuentes\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "C:\Users\MarceloFuentes\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asynci

# 4.2 Embedding

In [ ]:
from keras.models import Sequential
def embed_model(input_shape, output_sequence_length, english_vocab_size, quechua_vocab_size):
    learning_rate = 1e-3
    rnn = GRU(64, return_sequences=True, activation="tanh")
    
    embedding = Embedding(quechua_vocab_size, 64, input_length=input_shape[1]) 
    logits = TimeDistributed(Dense(quechua_vocab_size, activation="softmax"))
    
    model = Sequential()
    #em can only be used in first layer --> Keras Documentation
    model.add(embedding)
    model.add(rnn)
    model.add(logits)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model
tests.test_embed_model(embed_model)
tmp_x = pad(preproc_english_sentences, max_quechua_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_quechua_sentences.shape[-2]))
embeded_model = embed_model(
    tmp_x.shape,
    max_quechua_sequence_length,
    english_vocab_size,
    quechua_vocab_size)
embeded_model.fit(tmp_x, preproc_quechua_sentences, batch_size=1024, epochs=10, validation_split=0.2)
print(logits_to_text(embeded_model.predict(tmp_x[:1])[0], quechua_tokenizer))